In [ ]:
!pip install transformers
!pip install tensorboardX
!pip install wikipedia
!pip install swifter

In [ ]:
import torch
import tensorflow as tf
import pandas as pd
import wikipedia 
import swifter
import numpy as np
import nltk

In [ ]:
#Creating PyDrive instance to load in data from PeTaL shared drive, follow the steps to authenticate
!pip install -U -q PyDrive 
  
from pydrive.auth import GoogleAuth 
from pydrive.drive import GoogleDrive 
from google.colab import auth 
from oauth2client.client import GoogleCredentials 
  
  
# Authenticate and create the PyDrive client. 
auth.authenticate_user() 
gauth = GoogleAuth() 
gauth.credentials = GoogleCredentials.get_application_default() 
drive = GoogleDrive(gauth)

In [ ]:
labeled_df_link = 'https://drive.google.com/file/d/1MJDIPe1C0dFHIPWu0w18IEJhVk4Xbk2x/view?usp=sharing'
#'https://drive.google.com/file/d/1OZnAk64SPXfnaEzQFfhDJd6AX3dntIy9/view?usp=sharing'
labeled_id = labeled_df_link.split("/")[-2]
labeled_downloaded = drive.CreateFile({'id':labeled_id})  
labeled_downloaded.GetContentFile('single_label.csv') 
#'Biological-Strategies-Export-2020-October-01-1849 (1).csv'
labeled_df = pd.read_csv('single_label.csv')

In [ ]:
new_labeled_df_link = 'https://drive.google.com/file/d/1tKxbJeMlJU_Dh62Xqgqdi7ua-AQRFEM9/view'
new_labeled_id = new_labeled_df_link.split("/")[-2]
new_labeled_downloaded = drive.CreateFile({'id':new_labeled_id})  
new_labeled_downloaded.GetContentFile('labeled_abstracts_for_ML.csv') 
new_labeled_df = pd.read_csv('labeled_abstracts_for_ML.csv') 

In [ ]:
labeled_df = labeled_df[['id', 'Title', 'Living Systems', 'Sources_source_link', 'Functions', 'Wikipedia', 'pdf_links', 'single_label']]
labeled_df = labeled_df[labeled_df['Functions'].notnull( )]
labeled_df = labeled_df[labeled_df['Sources_source_link'].notnull()]

In [ ]:
new_labeled_df = new_labeled_df[['title', 'abstract', 'labels', 'doi']]
#new_labeled_df = new_labeled_df[labeled_df['labels'].notnull( )]
#new_labeled_df = new_labeled_df[labeled_df['abstracts'].notnull()]

In [ ]:
labeled_df

,id,Title,Living Systems,Sources_source_link,Functions,Wikipedia,pdf_links,single_label
0,2324,Beak design absorbs high-energy impacts,Ramphastos toco,http://dx.doi.org/10.1016/j.actamat.2005.04.04...,Manage impact,"The toco toucan (Ramphastos toco), also known ...",[],Manage impact
1,2362,Saliva regulates digestion,Heloderma suspectum,http://www.jbc.org/content/267/11/7402.abstract,Maintain homeostasis|Regulate cellular processes,"The Gila monster (Heloderma suspectum, HEE-lə...",[],Maintain homeostasis
2,2367,'Bombs' distract predators,Swima bombiviridis,http://dx.doi.org/10.1126/science.1172488,Transform radiant energy (light)|Send light si...,Swima bombiviridis is a worm species that live...,"['/content/325/5943.toc.pdf', '/content/sci/32...",Transform radiant energy (light)
3,2393,Organ generates electricity,Electrophorus electricus,https://epub.uni-regensburg.de/2108/,Modify electric charge|Transform electrical en...,"The electric eel (Electrophorus electricus, ot...",['http://www.uni-regensburg.de/publikationen/m...,Modify electric charge
4,2400,Wings generate lift,Apis mellifera,http://biomimetic.pbworks.com/f/Short-amplitud...,Move in/through gases,The western honey bee or European honey bee (A...,[],Move in/through gases
...,...,...,...,...,...,...,...,...
341,93260,Pheromones turn nematodes into pest-killing ma...,Nematoda,https://biblio.ugent.be/publication/1269676/fi...,"Capture, absorb, or filter organisms|Cooperate...","The nematodes (UK: NEM-ə-tohdz, US: NEEM- Gr...","['/articles/s41598-020-62817-y.pdf', '/article...","Capture, absorb, or filter organisms"
342,93283,Interaction with adults leads to faster nest b...,Taeniopygia guttata,https://academic.oup.com/beheco/article/31/4/8...,Physically assemble structure|Self-replicate,The zebra finch (Taeniopygia guttata) is the m...,request error,Physically assemble structure
343,93287,Chemicals in oregano act as fungicide,Origanum vulgare,"https://sci-hub.st/10.1111/1750-3841.12700,htt...",Chemically break down organic compounds|Distri...,"Oregano (US: , UK: ; Origanum vulgare) is a fl...",[],Chemically break down organic compounds
344,93338,Brain acts as both teacher and student,Taeniopygia guttata,"https://elifesciences.org/articles/20944,https...",Encode/Decode|Learn|Differentiate signal from ...,The zebra finch (Taeniopygia guttata) is the m...,[],Encode/Decode


In [ ]:
new_labeled_df

,title,abstract,labels,doi
0,Biomechanics of climbing palms and how they climb,Climbing plants have fascinated botanists sinc...,"Attach,Move",10.4161/psb.4.9.9426
1,Halogenated DOPA in a Marine Adhesive Protein,"The sandcastle worm Phragmatopoma californica,...","Attach,Physically assemble/disassemble",10.1080/00218460902782188
2,Probing the molecular and electronic structure...,Molecular structure and tautomeric/conformatio...,Protect from harm,10.1021/jp809138s
3,Pigment chemistry: the red sweat of the hippop...,"Within a few minutes of perspiration, the colo...",Protect from harm,10.1038/429363a
4,Most fertilizing mouse spermatozoa begin their...,"To fuse with oocytes, spermatozoa of eutherian...",Maintain ecological community,10.1073/pnas.1018202108
...,...,...,...,...
355,Rapid increase of vacuolar volume in response ...,Suspension-cultured cells of mangrove [Bruguie...,Process resources,10.1007/s00425-002-0878-2
356,How Delisea pulchra furanones affect quorum se...,Halogenated furanones produced by the benthic ...,Protect from harm,10.1099/00221287-146-12-3237
357,Evidence that halogenated furanones from Delis...,Acylated homoserine lactone (AHL)-mediated gen...,Protect from harm,10.1099/13500872-145-2-283
358,Human defensins: turning defense into offense?,Defensins are a family of antimicrobial cation...,Protect from harm,10.2174/187152607780090702


In [ ]:
# All labels in dataset sorted by frequency
labeled_df['single_label'] = labeled_df.apply(lambda x: x['single_label'].split('|')[0], axis=1)
labeled_df['single_label'].value_counts().index

Index(['Capture, absorb, or filter organisms',
       'Capture, absorb, or filter chemical entities', 'Attach temporarily',
       'Protect from microbes', 'Move in/on liquids', 'Manage impact',
       'Capture, absorb, or filter energy', 'Protect from temperature',
       'Capture, absorb, or filter liquids', 'Coordinate by self-organization',
       'Attach permanently', 'Physically assemble structure',
       'Move in/through gases', 'Distribute solids', 'Manage compression',
       'Distribute liquids', 'Cooperate within the same species',
       'Protect from light', 'Maintain homeostasis', 'Modify light/color',
       'Cooperate/compete between different species', 'Distribute gases',
       'Expel solids', 'Catalyze chemical breakdown',
       'Capture, absorb, or filter solids', 'Protect from animals',
       'Chemically assemble organic compounds',
       'Sense light (visible spectrum) from the environment',
       'Distribute energy', 'Break down living materials',
       'Ch

In [ ]:
# All labels in NEW dataset sorted by frequency
new_labeled_df['labels'] = new_labeled_df.apply(lambda x: x['labels'].split('|')[0], axis=1)
new_labeled_df['labels'].value_counts().index

Index(['Protect from harm', 'Move', 'Process information', 'Attach',
       'Maintain structural integrity', 'Process resources',
       'Chemically modify', 'Maintain ecological community',
       'Process information,Change size or color', 'Change size or color',
       'Maintain structural integrity,Protect from harm', 'Attach,Move',
       'Process resources,Protect from harm',
       'Physically assemble/disassemble,Protect from harm',
       'Physically assemble/disassemble', 'Attach,Protect from harm',
       'Protect from harm,Process information', 'Move,Process resources',
       'Maintain structural integrity,Move',
       'Protect from harm,Maintain ecological community',
       'Change size or color,Physically assemble/disassemble,Protect from harm',
       'Change size or color,Protect from harm',
       'Process information,Chemically modify', 'Process resources,Move',
       'Attach,Physically assemble/disassemble',
       'Maintain ecological community,Process informati

In [ ]:
# import re
# import string

# labels = []
# docs = []
# labels_test = []
# docs_test = []
# labels_dict = ['Capture, absorb, or filter organisms', 'Capture, absorb, or filter chemical entities', 'Protect from microbe', 'Move in/on liquids', 'Attach temporarily']

# single_label = labeled_df["single_label"].tolist()
# wikipedia = labeled_df["Wikipedia"].tolist()
# title = labeled_df["Title"].tolist()
# living_systems = labeled_df["Living Systems"].tolist()
# for i in range(len(title)):
#   if i < len(title) - 73: #310, 36
#     if single_label[i] == 'Capture, absorb, or filter organisms':
#       docs.append(wikipedia[i])
#       labels.append(labels_dict.index(single_label[i]))
#     if single_label[i] == 'Capture, absorb, or filter chemical entities':
#       docs.append(wikipedia[i])
#       labels.append(labels_dict.index(single_label[i]))
#     if single_label[i] == 'Protect from microbe':
#       docs.append(wikipedia[i])
#       labels.append(labels_dict.index(single_label[i]))
#     if single_label[i] == 'Move in/on liquids':
#       docs.append(wikipedia[i])
#       labels.append(labels_dict.index(single_label[i]))
#     if single_label[i] == 'Attach temporarily':
#       docs.append(wikipedia[i])
#       labels.append(labels_dict.index(single_label[i]))
#   else:
#     if single_label[i] == 'Capture, absorb, or filter organisms':
#       docs_test.append(wikipedia[i])
#       labels_test.append(labels_dict.index(single_label[i]))
#     if single_label[i] == 'Capture, absorb, or filter chemical entities':
#       docs_test.append(wikipedia[i])
#       labels_test.append(labels_dict.index(single_label[i]))
#     if single_label[i] == 'Protect from microbe':
#       docs_test.append(wikipedia[i])
#       labels_test.append(labels_dict.index(single_label[i]))
#     if single_label[i] == 'Move in/on liquids':
#       docs_test.append(wikipedia[i])
#       labels_test.append(labels_dict.index(single_label[i]))
#     if single_label[i] == 'Attach temporarily':
#       docs_test.append(wikipedia[i])
#       labels_test.append(labels_dict.index(single_label[i]))

# print ("Number of training labels: {:}".format(len(labels)))
# print ("Number of training docs: {:}".format(len(docs)))
# print ("Number of test labels: {:}".format(len(labels_test)))
# print ("Number of test docs: {:}".format(len(docs_test)))
# # print(labels)
# # print(docs)
# # print(labels_test)
# # print(docs_test)

Number of training labels: 51
Number of training docs: 51
Number of test labels: 9
Number of test docs: 9


In [ ]:
import re
import string

labels = []
docs = []
labels_test = []
docs_test = []
labels_dict = ['Protect from harm', 'Move', 'Process information', 'Attach', 'Maintain structural integrity']

title = new_labeled_df["title"].tolist()
abstract = new_labeled_df["abstract"].tolist()
labels = new_labeled_df["labels"].tolist()
doi = new_labeled_df["doi"].tolist()
for i in range(len(title)):
  if i < len(title) - 73: #310, 36
    if labels[i] == labels_dict[0]:
      docs.append(abstract[i])
      labels.append(labels_dict.index(labels[i]))
    if labels[i] == labels_dict[1]:
      docs.append(abstract[i])
      labels.append(labels_dict.index(labels[i]))
    if labels[i] == labels_dict[2]:
      docs.append(abstract[i])
      labels.append(labels_dict.index(labels[i]))
    if labels[i] == labels_dict[3]:
      docs.append(abstract[i])
      labels.append(labels_dict.index(labels[i]))
    if labels[i] == labels_dict[4]:
      docs.append(abstract[i])
      labels.append(labels_dict.index(labels[i]))
  else:
    if labels[i] == labels_dict[0]:
      docs_test.append(abstract[i])
      labels_test.append(labels_dict.index(labels[i]))
    if labels[i] == labels_dict[1]:
      docs_test.append(abstract[i])
      labels_test.append(labels_dict.index(labels[i]))
    if labels[i] == labels_dict[2]:
      docs_test.append(abstract[i])
      labels_test.append(labels_dict.index(labels[i]))
    if labels[i] == labels_dict[3]:
      docs_test.append(abstract[i])
      labels_test.append(labels_dict.index(labels[i]))
    if labels[i] == labels_dict[4]:
      docs_test.append(abstract[i])
      labels_test.append(labels_dict.index(labels[i]))

print ("Number of training labels: {:}".format(len(labels)))
print ("Number of training docs: {:}".format(len(docs)))
print ("Number of test labels: {:}".format(len(labels_test)))
print ("Number of test docs: {:}".format(len(docs_test)))
print(labels)
print(docs)
print(labels_test)
print(docs_test)

Number of training labels: 492
Number of training docs: 132
Number of test labels: 45
Number of test docs: 45
['Attach,Move', 'Attach,Physically assemble/disassemble', 'Protect from harm', 'Protect from harm', 'Maintain ecological community', 'Maintain structural integrity', 'Protect from harm', 'Protect from harm', 'Maintain structural integrity', 'Attach,Maintain structural integrity', 'Maintain structural integrity,Move', 'Chemically modify', 'Process resources', 'Process information,Change size or color', 'Attach', 'Protect from harm', 'Maintain ecological community,Process information', 'Protect from harm', 'Protect from harm', 'Attach,Move', 'Process information', 'Protect from harm', 'Change size or color', 'Maintain structural integrity', 'Physically assemble/disassemble', 'Protect from harm', 'Physically assemble/disassemble', 'Process resources', 'Protect from harm', 'Process information', 'Maintain structural integrity,Move', 'Maintain structural integrity,Move', 'Move,Prote

In [ ]:
## Preprocessing

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')

features = tfidf.fit_transform(docs.Consumer_complaint_narrative).toarray()
labels = docs.category_id
features.shape

AttributeError: ignored

In [ ]:
from nltk import word_tokenize

def tokenize(docs):
  

In [ ]:
# Ecnoding / TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.preprocessing import LabelEncoder

Encoder = LabelEncoder()
labels = Encoder.fit_transform(labels)
labels_test = Encoder.fit_transform(labels_test)
Tfidf_vect = TfidfVectorizer()
Tfidf_vect.fit(docs)
docs_Tfidf = Tfidf_vect.transform(docs)
docs_test_Tfidf = Tfidf_vect.transform(docs)

In [ ]:
print(Tfidf_vect.vocabulary_)

{'molecular': 2983, 'structure': 4445, 'and': 341, 'tautomeric': 4593, 'conformational': 1053, 'preferences': 3607, 'of': 3224, 'norhipposudoric': 3165, 'hipposudoric': 2230, 'acids': 170, 'the': 4647, 'recently': 3815, 'isolated': 2549, 'pigments': 3503, 'hippopotamus': 2229, 'amphibius': 319, 'sweat': 4547, 'were': 5042, 'investigated': 2521, 'using': 4905, 'density': 1303, 'functional': 2017, 'theory': 4655, 'dft': 1360, 'pbe0': 3393, 'formalism': 1972, 'among': 312, 'large': 2626, 'variety': 4929, 'possible': 3583, 'structures': 4447, 'two': 4829, 'similar': 4213, 'keto': 2587, 'enol': 1630, 'tautomer': 4592, 'conformers': 1054, 'are': 419, 'nearly': 3100, 'isoenergetic': 2545, 'markedly': 2807, 'more': 3003, 'stable': 4365, 'than': 4644, 'others': 3292, 'both': 666, 'in': 2362, 'gas': 2045, 'phase': 3455, 'aqueous': 408, 'solution': 4285, 'bulk': 708, 'solvent': 4289, 'effect': 1544, 'was': 5011, 'accounted': 154, 'for': 1963, 'with': 5083, 'polarizable': 3541, 'continuum': 1104, 

In [ ]:
print(docs_Tfidf)

  (0, 5083)	0.053777905560908834
  (0, 5058)	0.024024335253188603
  (0, 5042)	0.08703090173035329
  (0, 5011)	0.02839086195170911
  (0, 4929)	0.0537441765320726
  (0, 4916)	0.06136911124542241
  (0, 4910)	0.07081345165263402
  (0, 4909)	0.07081345165263402
  (0, 4905)	0.06607218004757519
  (0, 4862)	0.07081345165263402
  (0, 4829)	0.033036090023787594
  (0, 4739)	0.07081345165263402
  (0, 4724)	0.04373998453793101
  (0, 4712)	0.04640018585594644
  (0, 4682)	0.018209839268539873
  (0, 4672)	0.025169993208248483
  (0, 4655)	0.07081345165263402
  (0, 4647)	0.3981159449878784
  (0, 4644)	0.031431260466470476
  (0, 4593)	0.07081345165263402
  (0, 4592)	0.07081345165263402
  (0, 4547)	0.06136911124542241
  (0, 4506)	0.06528886667036966
  (0, 4447)	0.04335978734118508
  (0, 4445)	0.044299836124861006
  :	:
  (131, 1456)	0.05171657595217241
  (131, 1419)	0.05171657595217241
  (131, 1416)	0.06282572084762422
  (131, 1399)	0.05612815186269535
  (131, 1397)	0.05171657595217241
  (131, 1343)	0.062

In [ ]:
# Linear SVM
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, auc, precision_recall_curve
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer

In [ ]:
SVM = svm.SVC(kernel = 'linear')
SVM.fit(docs_Tfidf, labels)
print("training accuracy: ", SVM.score(docs_Tfidf, labels)*100)
print("testing accuracy: ", SVM.score(docs_test_Tfidf, labels_test)*100)

In [ ]:
#plotting top n features and their coefficients
plot_coefficients(SVM, Tfidf_vect.get_feature_names())

In [ ]:
# Optimization
tfidf = TfidfVectorizer()
clf = svm.SVC()

In [ ]:
pipe = Pipeline([
    ('tfidf', tfidf),
    ('clf', clf)
])

In [ ]:
linear_grid = {'tfidf__max_features': [500, 1000, 5000, None],
    'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'clf__C': np.logspace(-1, 3, 5),
    'clf__kernel' : ['linear']}
    
poly_grid = {'tfidf__max_features': [500, 1000, 5000, None],
    'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'clf__C': np.logspace(-1, 3, 5),
    'clf__kernel' : ['poly'],
    'clf__degree' : [2, 3],
    'clf__gamma': np.logspace(-2, 2, 5)}

rbf_grid = {'tfidf__max_features': [500, 1000, 5000, None],
    'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'clf__C': np.logspace(-1, 3, 5),
    'clf__kernel' : ['rbf'],
    'clf__gamma': np.logspace(-2, 2, 5)}

In [ ]:
scv_linear = RandomizedSearchCV(pipe, param_distributions = linear_grid, n_iter = 10, cv = 5, verbose = 2, n_jobs = -1, return_train_score = True, random_state = 42)
scv_linear.fit(Train_X, Train_Y);

In [ ]:
scv_poly = RandomizedSearchCV(pipe, param_distributions = poly_grid, n_iter = 10, cv = 5, verbose = 2, n_jobs = -1, return_train_score = True, random_state = 42)
scv_poly.fit(Train_X, Train_Y);

In [ ]:
scv_rbf = RandomizedSearchCV(pipe, param_distributions = rbf_grid, n_iter = 10, cv = 5, verbose = 2, n_jobs = -1, return_train_score = True, random_state = 42)
scv_rbf.fit(Train_X, Train_Y);

In [ ]:
scv_linear.best_params_
best_clf = scv_linear.best_estimator_

In [ ]:
print("SVM Training Accuracy Score -> ", best_clf.score(Train_X, Train_Y)*100)
print("SVM Testing Accuracy Score -> ", best_clf.score(Test_X, Test_Y)*100)

In [ ]:
Pred_Y = best_clf.predict(Test_X)
Score_Y = best_clf.decision_function(Test_X)